In [1]:
# 🛠️ Install dependencies (run this in a separate Colab cell first)
!pip install flask pyngrok nest_asyncio playwright pandas
!playwright install chromium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 18.7 MB/s eta 0:00:00
172.5 MiB [] 0% 0.0s172.5 MiB [] 0% 52.0s172.5 MiB [] 0% 25.3s172.5 MiB [] 0% 17.9s172.5 MiB [] 0% 9.2s172.5 MiB [] 1% 5.4s172.5 MiB [] 2% 4.2s172.5 MiB [] 3% 3.5s172.5 MiB [] 4% 3.1s172.5 MiB [] 4% 2.9s172.5 MiB [] 5% 3.0s172.5 MiB [] 6% 2.8s172.5 MiB [] 6% 2.7s172.5 MiB [] 8% 2.6s172.5 MiB [] 8% 2.5s172.5 MiB [] 9% 2.4s172.5 MiB [] 10% 2.4s172.5 MiB [] 11% 2.3s172.5 MiB [] 13% 2.2s172.5 MiB [] 13% 2.1s172.5 MiB [] 14% 2.1s172.5 MiB [] 15% 2.0s172.5 MiB [] 16% 1.9s172.5 MiB [] 18% 1.8s172.5 MiB [] 19% 1.8s172.5 MiB [] 20% 1.7s172.5 MiB [] 21% 1.7s172.5 MiB [] 22% 1.7s172.5 MiB [] 23% 1.6s172.5 MiB [] 24% 1.6s172.5 MiB [] 25% 1.5s172.5 MiB [] 26% 1.5s172.5 MiB [] 26% 1.6s172.5 MiB [] 27% 1.5s172.5 MiB [] 28% 1.5s172.5 MiB [] 30% 1.4s172.5 MiB [] 31% 1.4s172.5 MiB [] 33% 1.3s172.5 MiB [] 34% 1.2s172.5 MiB [] 36% 1.2s172.5 MiB [] 37% 1.2s172.5 MiB [] 38% 1.1s172.5 MiB [] 40% 1.1s172.5 MiB [] 41% 1.0s172.5 MiB 

In [ ]:
# ✅ app.py (Flask + Playwright)
import sqlite3
import asyncio
from flask import Flask, render_template_string, request, send_file
from pyngrok import ngrok
from playwright.async_api import async_playwright
import pandas as pd
import nest_asyncio
from google.colab import userdata

nest_asyncio.apply()
app = Flask(__name__)

# 🔐 Set ngrok authtoken from Colab secrets
NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# 🌐 Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel available at: {public_url}")

# 📄 HTML Template with dropdowns and validation
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head><title>District Court Search</title></head>
<body>
  <h2>Search Delhi District Court Case</h2>
  <form method="POST" onsubmit="return validateForm()">
    <label>District Court:</label>
    <select name="court_code" required>
      <option value="1">Tis Hazari (Central)</option>
      <option value="2">Patiala House (New Delhi)</option>
      <option value="3">Karkardooma (East)</option>
      <option value="4">Rohini (North-West)</option>
      <option value="5">Saket (South)</option>
      <option value="6">Dwarka (South-West)</option>
    </select><br><br>

    <label>Case Type: </label>
    <select name="case_type" required>
      <option value="CS">CS - Civil Suit</option>
      <option value="CC">CC - Criminal Complaint</option>
      <option value="SC">SC - Sessions Case</option>
      <option value="MAC">MAC - Motor Accident Claim</option>
      <option value="EX">EX - Execution</option>
      <option value="M">M - Miscellaneous</option>
    </select><br>

    <label>Case Number: </label><input name="case_number" required><br>
    <label>Filing Year: </label><input name="filing_year" required><br>
    <button type="submit">Search</button>
  </form>

  {% if result %}
    <h3>Results:</h3>
    <p><strong>Parties:</strong> {{ result.parties }}</p>
    <p><strong>Filing Date:</strong> {{ result.filing_date }}</p>
    <p><strong>Next Hearing:</strong> {{ result.next_hearing }}</p>
    <p><a href="{{ result.latest_order_link }}" target="_blank">Download Latest Order</a></p>
    <p><a href="/export" target="_blank">📥 Download All Logs (CSV)</a></p>
  {% elif error %}
    <p style="color:red;">{{ error }}</p>
  {% endif %}

<script>
  function validateForm() {
    const year = document.forms[0]["filing_year"].value;
    const yearPattern = /^[0-9]{4}$/;
    if (!yearPattern.test(year)) {
      alert("Please enter a valid 4-digit filing year.");
      return false;
    }
    return true;
  }
</script>
</body>
</html>
"""

# ✅ Log to SQLite
def log_to_db(case_type, case_number, filing_year, raw_html):
    conn = sqlite3.connect('queries.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            case_type TEXT,
            case_number TEXT,
            filing_year TEXT,
            raw_html TEXT
        )
    ''')
    c.execute('''
        INSERT INTO logs (case_type, case_number, filing_year, raw_html)
        VALUES (?, ?, ?, ?)
    ''', (case_type, case_number, filing_year, raw_html))
    conn.commit()
    conn.close()

# 🧠 Scraper Function: District Court Search
async def fetch_case(case_type, case_number, filing_year, court_code):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto("https://services.ecourts.gov.in/ecourtindia_v6/", timeout=60000)
        await page.click("text=Case Status")
        await page.wait_for_load_state("networkidle")

        # Delhi State = 7
        await page.select_option("select#sess_state_code", "7")
        await page.wait_for_timeout(1000)
        await page.select_option("select#sess_dist_code", court_code)
        await page.wait_for_timeout(1000)

        await page.click("input[value='C']")
        await page.wait_for_timeout(500)

        await page.select_option("select#search_case_type", case_type.upper())
        await page.fill("input#search_case_no", case_number)
        await page.fill("input#search_case_year", filing_year)
        await page.click("input#search_case_submit")
        await page.wait_for_load_state("networkidle")
        await page.wait_for_timeout(3000)

        html = await page.content()

        try:
            parties = await page.inner_text("xpath=//td[contains(text(),'Petitioner')]/following-sibling::td", timeout=3000)
            next_hearing = await page.inner_text("xpath=//td[contains(text(),'Next Hearing Date')]/following-sibling::td", timeout=3000)
            filing_date = await page.inner_text("xpath=//td[contains(text(),'Filing Date')]/following-sibling::td", timeout=3000)
        except:
            raise Exception("Case not found or layout changed.")

        result = {
            "parties": parties.strip(),
            "filing_date": filing_date.strip(),
            "next_hearing": next_hearing.strip(),
            "latest_order_link": "https://services.ecourts.gov.in"
        }

        await browser.close()
        return result, html

# 📥 Export all logs as CSV
@app.route('/export')
def export_csv():
    conn = sqlite3.connect('queries.db')
    df = pd.read_sql_query("SELECT * FROM logs", conn)
    conn.close()
    df.to_csv("queries.csv", index=False)
    return send_file("queries.csv", as_attachment=True)

# 🌐 Main route
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        court_code = request.form['court_code']
        case_type = request.form['case_type']
        case_number = request.form['case_number']
        filing_year = request.form['filing_year']
        try:
            result, html = asyncio.run(fetch_case(case_type, case_number, filing_year, court_code))
            log_to_db(case_type, case_number, filing_year, html)
            return render_template_string(HTML_TEMPLATE, result=result)
        except Exception as e:
            return render_template_string(HTML_TEMPLATE, error=str(e))
    return render_template_string(HTML_TEMPLATE)

# ▶️ Run the app
app.run()


 * ngrok tunnel available at: NgrokTunnel: "https://734bb4199924.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Aug/2025 07:09:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Aug/2025 07:09:24] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Aug/2025 07:09:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Aug/2025 07:09:50] "GET /favicon.ico HTTP/1.1" 404 -
